In [17]:
%load_ext autoreload
%autoreload 2

In [18]:
import numpy as np
import os
import networkx as nx
import subprocess
import glob
from pathlib import Path
from matplotlib import pyplot as plt
from matplotlib.widgets import Slider
from ipywidgets import interact, FloatSlider

In [19]:
dataset_name = 'ToH'

feature_type = 'sift_default'
matching_type = 'sift_default'

# feature_type = 'superpoint'
# matching_type = 'superglue'

geometric_verification_type = 'default'
colmap_path = 'colmap'

In [20]:
os.environ['MKL_THREADING_LAYER'] = 'GNU'
output1 = subprocess.run(['python', '../scripts/match_features.py',
                         '--dataset_name', dataset_name,
                         '--feature_type', feature_type,
                         '--matching_type', matching_type,
                         '--geometric_verification_type', geometric_verification_type,
                         '--colmap_path', colmap_path,
                         '--use_gpu'], capture_output=True)

In [21]:
# print(str(output1.stderr, 'utf-8'))
# print(str(output1.stdout, 'utf-8'))

In [22]:
track_degree = '3'

# The coverage controls how many iconic images will be selected. As for small-scale indoor scenes, a large value between 0.7 and 0.9 is recommended; otherwise, for large-scale unstructured datasets, the value around 0.6 would be enough.
coverage_thres = '0.9' 
# coverage_thres = '0.6'

alpha = '0.1'
minimal_views = '5'
ds = 'largearray'

In [36]:
output2 = subprocess.run(['python', '../scripts/disambiguate_yan.py',
                         '--dataset_name', dataset_name,
                         '--feature_type', feature_type,
                         '--matching_type', matching_type,
                         '--geometric_verification_type', geometric_verification_type,
                         '--track_degree', track_degree,
                         '--coverage_thres', coverage_thres,
                         '--alpha', alpha,
                         '--minimal_views', minimal_views,
                         '--ds', ds], capture_output=True)

In [24]:
# print(str(output2.stdout, 'utf-8'))

In [37]:
root = Path.cwd().parents[0]
dataset_path_heinly = root / 'datasets' / 'heinly2014'
dataset_names_heinly = [
    path.split('/')[-1]
    for path in glob.glob(str(dataset_path_heinly / "*/"))
]
dataset_path_yan = root / 'datasets' / 'yan2017'
dataset_names_yan = [
    path.split('/')[-1] for path in glob.glob(str(dataset_path_yan / "*/"))
]
if dataset_name in dataset_names_heinly:
    dataset_folder = 'heinly2014'
elif dataset_name in dataset_names_yan:
    dataset_folder = 'yan2017'
else:
    print(f"Unknown dataset name: {dataset_name}")
    raise ValueError
    
results_path = (root / 'results' / dataset_name /
                '_'.join([feature_type, matching_type, geometric_verification_type]))
scores_name = 'scores' + f'_yan_t{track_degree}_c{coverage_thres}_a{alpha}_m{minimal_views}.npy'
scores = np.load(results_path / scores_name)

In [38]:
@interact(thres=FloatSlider(value=0.1, min=0.0, max=0.5, step=0.01))
def plot(thres):
    valid_scores = scores.copy()
    valid_scores[scores < thres] = 0.
    graph = nx.from_numpy_matrix(valid_scores)
    pos = nx.nx_agraph.graphviz_layout(graph)
    edge_vmin = np.percentile(valid_scores[valid_scores.nonzero()], 10)
    edge_vmax = np.percentile(valid_scores[valid_scores.nonzero()], 90)
    nx.draw(graph, pos, with_labels=True, edge_cmap=plt.cm.YlOrRd,
            edge_color=[graph[u][v]['weight'] for u, v in graph.edges],
            edge_vmin=edge_vmin, edge_vmax=edge_vmax)
    plt.show()

interactive(children=(FloatSlider(value=0.1, description='thres', max=0.5, step=0.01), Output()), _dom_classes…

In [39]:
scores_name = 'scores' + f'_yan_t{track_degree}_c{coverage_thres}_a{alpha}_m{minimal_views}.npy'
filter_type = 'threshold'
threshold = '0.05' # small scene 0.3, large scene 0.04 to 0.1

In [40]:
output3 = subprocess.run(['python', '../scripts/filter_matches.py',
                         '--dataset_name', dataset_name,
                         '--feature_type', feature_type,
                         '--matching_type', matching_type,
                         '--geometric_verification_type', geometric_verification_type,
                         '--colmap_path', colmap_path,
                         '--scores_name', scores_name,
                         '--filter_type', filter_type,
                         '--threshold', threshold,
                         '--reconstruct_unfiltered',
                         '--reconstruct_filtered'], capture_output=True)

In [29]:
# print(str(output3.stderr, 'utf-8'))
# print(str(output3.stdout, 'utf-8'))

In [41]:
from disambiguation.utils import visualize_model

In [42]:
backbone = f'_yan_t{track_degree}_c{coverage_thres}_a{alpha}_m{minimal_views}'
input_model = results_path / ('sparse' + backbone + f'_t{threshold}') / '0'
input_format = '.bin'
visualize_model.main(input_model, input_format)

num_cameras: 1
num_images: 338
num_points3D: 215793
